### Manipulation of PeaskDB de novo-assisted database search results of Trocas 8 (low water, November 2019) non incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 675) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/684_Trocas8_June2021_PEAKS_143/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/684_Trocas8_June2021_PEAKS_143


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 684_Trocas8_June2021_PEAKS_143_peptide.csv     'de novo only peptides.csv'
 684_Trocas8_June2021_PEAKS_143_proteins.csv     peptides_1_1_0.mzid
 684_Trocas8_June2021_PEAKS_143_proteins.fasta   protein-peptides.csv
'DB search psm.csv'


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup684 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/684_Trocas8_June2021_PEAKS_143/684_Trocas8_June2021_PEAKS_143_peptide.csv")

# remove redundant rows
peaksdb684 = pd.DataFrame.drop_duplicates(peaksdbdup684)

print(peaksdb684.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb684.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb684['#Spec']
del peaksdb684['Accession']
del peaksdb684['PTM']
del peaksdb684['AScore']

mean_length = peaksdb684['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup684))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb684))

#look at the dataframe
peaksdb684.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Sample 10', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Sample 10', 'Accession', 'PTM', 'AScore'],
      dtype='object')
8.833333333333334
# redundant peaksdb peptides in combined dataframe 36
# nonredundant peaksdb peptides in combined dataframe 36


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,LSSPATLNSR,45.95,1044.5564,10,0.4,523.2857,44.08,5700000.0,10,7173,20210624_Trocas8_684_Baylique57_surface_DDA_12...
1,LATVISPR,38.28,855.5178,8,-0.7,428.7658,58.95,2930000.0,10,9710,20210624_Trocas8_684_Baylique57_surface_DDA_12...
2,LATVLSPR,38.28,855.5178,8,-0.7,428.7658,58.95,2930000.0,10,9710,20210624_Trocas8_684_Baylique57_surface_DDA_12...
3,YLYEIAR,31.22,926.4861,7,-0.2,464.2502,68.48,20300.0,10,11144,20210624_Trocas8_684_Baylique57_surface_DDA_12...
4,YLYELAR,31.22,926.4861,7,-0.2,464.2502,68.48,20300.0,10,11144,20210624_Trocas8_684_Baylique57_surface_DDA_12...


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb684['A'] = peaksdb684['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb684['C'] = peaksdb684['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb684['D'] = peaksdb684['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb684['E'] = peaksdb684['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb684['F'] = peaksdb684['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb684['G'] = peaksdb684['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb684['H'] = peaksdb684['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb684 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb684['I'] = peaksdb684['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb684['K'] = peaksdb684['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb684['L'] = peaksdb684['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb684['M'] = peaksdb684['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb684['N'] = peaksdb684['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb684['P'] = peaksdb684['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb684['Q'] = peaksdb684['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb684['R'] = peaksdb684['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb684['S'] = peaksdb684['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb684['T'] = peaksdb684['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb684['V'] = peaksdb684['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb684['W'] = peaksdb684['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb684['Y'] = peaksdb684['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb684['c-carb'] = peaksdb684['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb684['m-oxid'] = peaksdb684['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb684['n-deam'] = peaksdb684['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb684['q-deam'] = peaksdb684['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb684['stripped_peptide'] = peaksdb684['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb684['stripped_length'] = peaksdb684['stripped_peptide'].apply(len)

peaksdb684['NAAF_num.'] = peaksdb684['Area'] / peaksdb684['stripped_length']

# total the number of modifications in sequence
peaksdb684['ptm-total'] = peaksdb684['c-carb'] + peaksdb684['m-oxid'] + peaksdb684['n-deam'] + peaksdb684['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb684['stripped_IL']= peaksdb684['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb684.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/676_BY_surf_GF75_PDB/684_BY_surf_GF75_PDB.csv")

# check out the results
peaksdb684.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,LSSPATLNSR,45.95,1044.5564,10,0.4,523.2857,44.08,5700000.0,10,7173,...,0,0,0,0,0,LSSPATLNSR,10,570000.0,0,LSSPATLNSR
1,LATVISPR,38.28,855.5178,8,-0.7,428.7658,58.95,2930000.0,10,9710,...,0,0,0,0,0,LATVISPR,8,366250.0,0,LATVLSPR
2,LATVLSPR,38.28,855.5178,8,-0.7,428.7658,58.95,2930000.0,10,9710,...,0,0,0,0,0,LATVLSPR,8,366250.0,0,LATVLSPR
3,YLYEIAR,31.22,926.4861,7,-0.2,464.2502,68.48,20300.0,10,11144,...,2,0,0,0,0,YLYEIAR,7,2900.0,0,YLYELAR
4,YLYELAR,31.22,926.4861,7,-0.2,464.2502,68.48,20300.0,10,11144,...,2,0,0,0,0,YLYELAR,7,2900.0,0,YLYELAR


In [6]:
# keep only stripped peptide I/L and NAAF
db_684 = peaksdb684[['stripped_IL', 'Area', 'NAAF_num.']]

db_684.set_index('stripped_IL')

# write modified dataframe to new txt file
db_684.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/684_BY_surf_GF75_PDB_ILnaafs.csv")

db_684.head()

,stripped_IL,Area,NAAF_num.
0,LSSPATLNSR,5700000.0,570000.0
1,LATVLSPR,2930000.0,366250.0
2,LATVLSPR,2930000.0,366250.0
3,YLYELAR,20300.0,2900.0
4,YLYELAR,20300.0,2900.0


### Exporting txt files of stripped peptides at confidence cutoffs:

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep684moddup = peaksdb684[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep684dup = peaksdb684[["stripped_peptide"]]

# deduplicate both of these lists
pep684mod = pep684moddup.drop_duplicates()
pep684 = pep684dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 684:", len(pep684moddup))
print("Deduplicated modified PeaksDB peptides in 684:", len(pep684mod))
print("Total stripped PeaksDB peptides in 684:", len(pep684dup))
print("Deduplicated stripped PeaksDB peptides in 684:", len(pep684))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep684.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/676_BY_surf_GF75_PDB/684_BY_surf_GF75_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/676_BY_surf_GF75_PDB/684_BY_surf_GF75_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/676_BY_surf_GF75_PDB/684_BY_surf_GF75_PDB_stripped_peptides.fas

# look at the stripped peptides
pep684.head()

Total modified PeaksDB peptides in 684: 36
Deduplicated modified PeaksDB peptides in 684: 36
Total stripped PeaksDB peptides in 684: 36
Deduplicated stripped PeaksDB peptides in 684: 36


,stripped_peptide
0,LSSPATLNSR
1,LATVISPR
2,LATVLSPR
3,YLYEIAR
4,YLYELAR


## NAAF correction and exporting files with AA and PTM totals:

In [9]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb684['A'].sum(),
        'C': peaksdb684['C'].sum(),
        'D': peaksdb684['D'].sum(),
        'E': peaksdb684['E'].sum(),
        'F': peaksdb684['F'].sum(),
        'G': peaksdb684['G'].sum(),
        'H': peaksdb684['H'].sum(),
        'I': peaksdb684['I'].sum(),
        'K': peaksdb684['K'].sum(),
        'L': peaksdb684['L'].sum(),
        'M': peaksdb684['M'].sum(),
        'N': peaksdb684['N'].sum(),
        'P': peaksdb684['P'].sum(),
        'Q': peaksdb684['Q'].sum(),
        'R': peaksdb684['R'].sum(),
        'S': peaksdb684['S'].sum(),
        'T': peaksdb684['T'].sum(),
        'V': peaksdb684['V'].sum(),
        'W': peaksdb684['W'].sum(),
        'Y': peaksdb684['Y'].sum(),
        'c-carb': peaksdb684['c-carb'].sum(),
        'm-oxid': peaksdb684['m-oxid'].sum(),
        'n-deam': peaksdb684['n-deam'].sum(),
        'q-deam': peaksdb684['q-deam'].sum(),
        'Total area': peaksdb684['Area'].sum(),
        'Total length': peaksdb684['stripped_length'].sum()
       }

totalpeaksdb684 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb684['% C w/ carb'] = totalpeaksdb684['c-carb'] / totalpeaksdb684['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb684['% M w/ oxid'] = totalpeaksdb684['m-oxid'] / totalpeaksdb684['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb684['% N w/ deam'] = totalpeaksdb684['n-deam'] / totalpeaksdb684['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb684['% Q w/ deam'] = totalpeaksdb684['q-deam'] / totalpeaksdb684['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb684['NAAF denom.'] = totalpeaksdb684['Total area'] / totalpeaksdb684['Total length']

# write modified dataframe to new txt file
totalpeaksdb684.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/676_BY_surf_GF75_PDB/684_BY_surf_GF75_PDB_totals.csv")

totalpeaksdb684.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,26,2,7,6,11,22,4,24,22,31,...,0,1,1,24252028.0,315,1.0,0.0,0.066667,0.166667,76990.565079


In [10]:
# use the calculated NAAF factor (in totalpeaksdb684 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 76990.565079

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb684['NAAF factor'] = (peaksdb684['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb684_NAAF = peaksdb684[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb684_NAAF['A-NAAF20'] = peaksdb684_NAAF['A'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['C-NAAF20'] = peaksdb684_NAAF['C'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['D-NAAF20'] = peaksdb684_NAAF['D'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['E-NAAF20'] = peaksdb684_NAAF['E'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['F-NAAF20'] = peaksdb684_NAAF['F'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['G-NAAF20'] = peaksdb684_NAAF['G'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['H-NAAF20'] = peaksdb684_NAAF['H'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['I-NAAF20'] = peaksdb684_NAAF['I'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['K-NAAF20'] = peaksdb684_NAAF['K'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['L-NAAF20'] = peaksdb684_NAAF['L'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['M-NAAF20'] = peaksdb684_NAAF['M'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['N-NAAF20'] = peaksdb684_NAAF['N'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['P-NAAF20'] = peaksdb684_NAAF['P'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['Q-NAAF20'] = peaksdb684_NAAF['Q'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['R-NAAF20'] = peaksdb684_NAAF['R'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['S-NAAF20'] = peaksdb684_NAAF['S'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['T-NAAF20'] = peaksdb684_NAAF['T'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['V-NAAF20'] = peaksdb684_NAAF['V'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['W-NAAF20'] = peaksdb684_NAAF['W'] * peaksdb684['NAAF factor']
peaksdb684_NAAF['Y-NAAF20'] = peaksdb684_NAAF['Y'] * peaksdb684['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb684_NAAF['ccarb-NAAF20'] = peaksdb684_NAAF['c-carb'] * peaksdb684_NAAF['NAAF factor']
peaksdb684_NAAF['moxid-NAAF20'] = peaksdb684_NAAF['m-oxid'] * peaksdb684_NAAF['NAAF factor']
peaksdb684_NAAF['ndeam-NAAF20'] = peaksdb684_NAAF['n-deam'] * peaksdb684_NAAF['NAAF factor']
peaksdb684_NAAF['qdeam-NAAF20'] = peaksdb684_NAAF['q-deam'] * peaksdb684_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb684_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/676_BY_surf_GF75_PDB/684_BY_surf_GF75_PDB_naaf.csv")

peaksdb684_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,LSSPATLNSR,7.403505,1,0,0,0,0,0,0,0,...,7.403505,22.210514,7.403505,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,LATVISPR,4.757076,1,0,0,0,0,0,0,1,...,4.757076,4.757076,4.757076,4.757076,0.0,0.000000,0.0,0.0,0.0,0.0
2,LATVLSPR,4.757076,1,0,0,0,0,0,0,0,...,4.757076,4.757076,4.757076,4.757076,0.0,0.000000,0.0,0.0,0.0,0.0
3,YLYEIAR,0.037667,1,0,0,1,0,0,0,1,...,0.037667,0.000000,0.000000,0.000000,0.0,0.075334,0.0,0.0,0.0,0.0
4,YLYELAR,0.037667,1,0,0,1,0,0,0,0,...,0.037667,0.000000,0.000000,0.000000,0.0,0.075334,0.0,0.0,0.0,0.0


In [11]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb684 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb684_NAAF['NAAF factor'].sum(),
        'A': peaksdb684_NAAF['A-NAAF20'].sum(),
        'C': peaksdb684_NAAF['C-NAAF20'].sum(),
        'D': peaksdb684_NAAF['D-NAAF20'].sum(),
        'E': peaksdb684_NAAF['E-NAAF20'].sum(),
        'F': peaksdb684_NAAF['F-NAAF20'].sum(),
        'G': peaksdb684_NAAF['G-NAAF20'].sum(),
        'H': peaksdb684_NAAF['H-NAAF20'].sum(),
        'I': peaksdb684_NAAF['I-NAAF20'].sum(),
        'K': peaksdb684_NAAF['K-NAAF20'].sum(),
        'L': peaksdb684_NAAF['L-NAAF20'].sum(),
        'M': peaksdb684_NAAF['M-NAAF20'].sum(),
        'N': peaksdb684_NAAF['N-NAAF20'].sum(),
        'P': peaksdb684_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb684_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb684_NAAF['R-NAAF20'].sum(),
        'S': peaksdb684_NAAF['S-NAAF20'].sum(),
        'T': peaksdb684_NAAF['T-NAAF20'].sum(),
        'V': peaksdb684_NAAF['V-NAAF20'].sum(),
        'W': peaksdb684_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb684_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb684_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb684_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb684_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb684_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb684_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb684_NAAF['% C w/ carb.'] = totalpeaksdb684_NAAF['c-carb'] / totalpeaksdb684_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb684_NAAF['% M w/ oxid'] = totalpeaksdb684_NAAF['m-oxid'] / totalpeaksdb684_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb684_NAAF['% N w/ deam'] = totalpeaksdb684_NAAF['n-deam'] / totalpeaksdb684_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb684_NAAF['% Q w/ deam'] = totalpeaksdb684_NAAF['q-deam'] / totalpeaksdb684_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb684_NAAF['NAAF check'] = totalpeaksdb684_NAAF['NAAF'] / 76990.565079

# write modified dataframe to new txt file, same name + totals
totalpeaksdb684_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/676_BY_surf_GF75_PDB/684_BY_surf_GF75_PDB_naaf_totals.csv")

totalpeaksdb684_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,37.698238,29.508716,0.044334,0.30382,0.320418,0.392126,0.482123,0.21756,11.489698,0.354383,...,0.264533,0.044334,0.0,0.062581,0.0,1.0,0.0,0.007839,0.0,0.00049
